In [1]:
# Incluir las bibliotecas requeridas para el modelo y su métrica
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.neural_network import MLPClassifier

#Cargar el archivo desde el repositorio
# URL del archivo CSV público en S3
url = 'https://diabetes-dvcstore.s3.us-east-1.amazonaws.com/files/md5/10/1cada3906fab160ea188043d7f9a1b'
# Carga el archivo directamente en un DataFrame de pandas
df = pd.read_csv(url)

# Separar las características (X) y la variable objetivo (y)

X = df.drop(columns=['Diabetes_012'])
y = df['Diabetes_012']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [3]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [4]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio
import mlflow
import mlflow.sklearn

# defina el servidor para llevar el registro de modelos y artefactos
#mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("Diabetes_USA")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas.
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo.
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # defina los parámetros del modelo
    hidden_layer_sizes=(300,)
    max_iter=600
    learning_rate_init=0.001
    random_state=64
    # Cree el modelo con los parámetros definidos y entrénelo
    NN_model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=learning_rate_init,
                             max_iter=max_iter, random_state=random_state)
    NN_model.fit(X_train, y_train)

    # Realice predicciones de prueba
    predictions = NN_model.predict(X_test)

    # Registre los parámetros
    mlflow.log_param("hidden_layer_sizes", hidden_layer_sizes)
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("learning_rate_init", learning_rate_init)
    mlflow.log_param("random_state", random_state)

    # Registre el modelo
    mlflow.sklearn.log_model(NN_model, "Neuronal-Network-model")

    # Cree y registre la métrica de interés
    #mse = mean_squared_error(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    mlflow.log_metric("Precision", accuracy)
    mlflow.log_metric("Sensibilidad", recall)
    print(accuracy)
    print(recall)

2024/11/11 03:39:03 INFO mlflow.tracking.fluent: Experiment with name 'Diabetes_USA' does not exist. Creating a new experiment.
2024/11/11 03:39:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


0.7452508805358277
0.8475258918296893
